# Low power person detection on UAVs

This is the notebook with holds the complete pipeline for our project 

## We start with loading different models 
At the start of the project we used models from EfficientDet, Fomo, Yolo and mobilenet_ssd. After comparison we decided to only move forward with the YOLO model, therefore later code is written only for the yolo architecture

In [1]:
from ultralytics import YOLO
import tensorflow as tf
import tensorflow_hub as hub
import os


E0000 00:00:1761035375.384529    9204 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761035375.411541    9204 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761035375.571928    9204 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761035375.571969    9204 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761035375.571972    9204 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761035375.571975    9204 computation_placer.cc:177] computation placer already registered. Please check linka

In [2]:
# load_model.py

def load_yolo(model_name : str, model_name_ext: str):
    """
    Loads a YOLO modle
    """
    os.makedirs("models", exist_ok=True)

    model = YOLO(model_name_ext)
    exported_path = model.export(format="saved_model")


    return exported_path


def load_mobilenet_ssd(model_name: str, model_url: str = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"):
    """Loads MobileNet SSD from TensorFlow Hub"""
    os.makedirs("models", exist_ok=True)
    
    model = hub.load(model_url)
    saved_model_path = f"{model_name}_saved_model"
    tf.saved_model.save(model, saved_model_path)
    return saved_model_path


def load_efficientdet(model_name: str, model_url: str = "https://tfhub.dev/tensorflow/efficientdet/d0/1"):
    """Loads EfficientDet from TensorFlow Hub"""
    os.makedirs("models", exist_ok=True)

    model = hub.load(model_url)
    saved_model_path = f"{model_name}_saved_model"
    tf.saved_model.save(model, saved_model_path)
    return saved_model_path

In [ ]:

# download the yolo model
model_name = "yolo11n"
model_name_ext = "yolo11n.pt"
yolo_saved_model_path = load_yolo(model_name, model_name_ext)




Ultralytics 8.3.208 🚀 Python-3.12.3 torch-2.8.0+cu128 CPU (AMD Ryzen 7 7735U with Radeon Graphics)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)

TensorFlow SavedModel: starting export with tensorflow 2.19.0...
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /home/jakub/Documents/semester5/TinyML/uav-person/low-power-person-detection-uav-sar/src/calibration_image_sample_data_20x128x128x3_float32.npy...: 100% ━━━━━━━━━━━━ 1/1 77.1files/s 0.0s

ONNX: starting export with onnx 1.19.0 opset 22...
ONNX: slimming with onnxslim 0.1.71...
ONNX: export success ✅ 1.6s, saved as 'yolo11n.onnx' (10.2 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.2...
Saved artifact at 'yolo11n_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(

I0000 00:00:1761035395.241985    9204 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1761035395.242117    9204 single_machine.cc:374] Starting new session
W0000 00:00:1761035395.752249    9204 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1761035395.752267    9204 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1761035396.260982    9204 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1761035396.261154    9204 single_machine.cc:374] Starting new session
W0000 00:00:1761035396.711216    9204 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1761035396.711240    9204 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


TensorFlow SavedModel: export success ✅ 12.2s, saved as 'yolo11n_saved_model' (25.7 MB)

Export complete (12.7s)
Results saved to /home/jakub/Documents/semester5/TinyML/uav-person/low-power-person-detection-uav-sar/src
Predict:         yolo predict task=detect model=yolo11n_saved_model imgsz=640  
Validate:        yolo val task=detect model=yolo11n_saved_model imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


Now we downloaded yolo 11. ... and saved and ... and now we train

In [ ]:
#train.py

now we have the model and will optimize

In [ ]:


def optimize_model(model_name: str, model_path: str, output_dir: str = "models/optimized_models"):
    """Generate different variants of optimized models"""
    

    os.makedirs(output_dir, exist_ok=True)
    
    # Size optimized variants 
    _convert(model_name, model_path, output_dir, "size_float32", tf.lite.Optimize.OPTIMIZE_FOR_SIZE, None)
    _convert(model_name, model_path, output_dir, "size_float16", tf.lite.Optimize.OPTIMIZE_FOR_SIZE, tf.float16)
    _convert(model_name, model_path, output_dir, "size_dynamic", tf.lite.Optimize.OPTIMIZE_FOR_SIZE, "dynamic")
    


    # Latency optimized variants
    _convert(model_name, model_path, output_dir, "latency_float32", tf.lite.Optimize.OPTIMIZE_FOR_LATENCY, None)
    _convert(model_name, model_path, output_dir, "latency_float16", tf.lite.Optimize.OPTIMIZE_FOR_LATENCY, tf.float16)
    _convert(model_name, model_path, output_dir, "latency_dynamic", tf.lite.Optimize.OPTIMIZE_FOR_LATENCY, "dynamic")




def _convert(model_name: str, model_path: str, output_dir: str, suffix: str, optimization, quant_type):
    """Helper function to convert model with specific settings"""


    converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
    converter.optimizations = [optimization]

    # restrict to tensorflow builtin ops
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

    # float16 quantization
    if quant_type == tf.float16:
        converter.target_spec.supported_types = [tf.float16]
        converter.inference_input_type = tf.float16
        converter.inference_output_type = tf.float16
    # try dynamic range quantization
    elif quant_type == "dynamic":
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.inference_input_type = tf.float32
        converter.inference_output_type = tf.float32
    else:
        converter.inference_input_type = tf.float32
        converter.inference_output_type = tf.float32

    # try with experimental converter
    converter.experimental_new_converter = True

    # custom ops (not neccessary i think)
    converter.allow_custom_ops = True


    try:
        tflite_model = converter.convert()

    except ValueError as e:

        print(f"Skipping {suffix} for {model_name}: {e}")

        return None

    output_path = os.path.join(output_dir, f"{model_name}_{suffix}.tflite")

    with open(output_path, 'wb') as f:

        f.write(tflite_model)


    print(f"{suffix}: {output_path} ({len(tflite_model) / (1024*1024):.2f} MB)")


    return output_path




In [ ]:
# some calls for optimization

now benchmarking

In [ ]:
# benchamrking